In [ ]:
import os
import setGPU
# edit depending on where Vivado is installed:
# os.environ['PATH'] = '/<Xilinx installation directory>/Vivado/<version>/bin:' + os.environ['PATH']
os.environ['PATH'] = '/xilinx/Vivado/2019.1/bin:' + os.environ['PATH']
import tensorflow as tf
from qkeras.utils import _add_supported_quantized_objects
import hls4ml
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu

from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model

In [ ]:
def yaml_load(config):
    with open(config) as stream:
        param = yaml.safe_load(stream)
        

def print_dict(d, indent=0):
    align=20
    for key, value in d.items():
        print('  ' * indent + str(key), end='')
        if isinstance(value, dict):
            print()
            print_dict(value, indent+1)
        else:
            print(':' + ' ' * (20 - len(key) - 2 * indent) + str(value))

In [ ]:
model_file_path = 'model_QInteractionNetwork_nconst_8_nbits_8.h5'

In [ ]:
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects
co = {}
_add_supported_quantized_objects(co)    

model = load_model(model_file_path, custom_objects=co)
model.summary()

In [ ]:
import hls4ml
config = hls4ml.utils.config_from_keras_model(model, granularity='name')

config['Model'] = {}
config['Model']['ReuseFactor'] = 1
config['Model']['Strategy'] = 'Latency'
config['Model']['Precision'] = 'ap_fixed<16,6>'
config['SkipOptimizers'] = ['optimize_pointwise_conv']
for layer in config['LayerName'].keys():
    config['LayerName'][layer]['Trace'] = True
config['LayerName']['q_activation_3']['Precision']['result'] = 'ap_fixed<8,3,AP_RND,AP_SAT>'
config['LayerName']['q_activation_6']['Precision']['result'] = 'ap_fixed<8,3,AP_RND,AP_SAT>'
#hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
#hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
#hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

cfg = hls4ml.converters.create_vivado_config(fpga_part='xc7z020clg400-1')
cfg['HLSConfig'] = config
cfg['IOType'] = 'io_parallel'
cfg['Backend'] = 'Vivado'
cfg['ClockPeriod'] = 10
cfg['KerasModel'] = model
cfg['OutputDir'] = 'hls_output'

print("-----------------------------------")
print_dict(cfg)
print("-----------------------------------")

hls_model = hls4ml.converters.keras_to_hls(cfg)
hls_model.compile()

In [ ]:
import numpy as np
import h5py
import os

#Data PATH
TEST_PATH = '/xilinx/scratch/hls4ml_LHCjet_150p_val/'

first=True
for file in os.listdir(TEST_PATH):
  print("Appending %s" %file)

  with h5py.File(TEST_PATH+file, 'r') as data:
    if first : 
        first=False
        jetConstituent = data['jetConstituentList'][:,:,[5,8,11]]
        target = data['jets'][:,-6:-1]
        featurenames = data.get('jetFeatureNames')
        featurenames = data.get('particleFeatureNames')
        images = data.get('jetImage')

    else:
        jetConstituent = np.concatenate( [ jetConstituent, data['jetConstituentList'][:,:,[5,8,11]] ] , axis=0 )
        target   = np.concatenate( [ target, data['jets'][:,-6:-1] ] , axis=0 )
                                    
print("Target shape =", target.shape)
print("Jet Constituents shape =", jetConstituent.shape)

In [ ]:
from einops import rearrange, reduce, repeat

# The dataset is N_jets x N_constituents x N_features
njet     = jetConstituent.shape[0]
nconstit = jetConstituent.shape[1]
nfeat    = jetConstituent.shape[2]

# Filter out constituents with Pt<2GeV
Ptmin = 2. 
constituents = np.zeros((njet, nconstit, nfeat) , dtype=np.float32) 
ij=0
max_constit=0
for j in range(njet):
    ic=0
    for c in range(nconstit):
        if ( jetConstituent[j,c,0] < Ptmin ):
            continue
        constituents[ij,ic,:] = jetConstituent[j,c,:] 
        ic+=1
    if (ic > 0):
        if ic > max_constit: max_constit=ic
        target[ij,:]=target[j,:] # assosicate the correct target a given graph 
        ij+=1

# Resizes the jets constituents and target arrays        
jetConstituent = constituents[0:ij,0:max_constit,:]
target = target[0:ij,:]

# Restric the number of constituents to a maximum of NMAX
nmax = 8
jetConstituent = jetConstituent[:,0:nmax,:]

# The dataset is N_jets x N_constituents x N_features
njet     = jetConstituent.shape[0]
nconstit = jetConstituent.shape[1]
nfeat    = jetConstituent.shape[2]


print('Number of jets =',njet)
print('Number of constituents =',nconstit)
print('Number of features =',nfeat)


# Shuffles Jet Constituents because L1 constituents are not Pt ordered to begin with
print("Before --->> jetConstituent[0,0:4,0] = ",jetConstituent[0,0:4,0])
jetConstituent = jetConstituent[ : , np.random.permutation(nconstit), : ]
print("After --->> jetConstituent[0,0:4,0] = ",jetConstituent[0,0:4,0])

In [ ]:
njet = 10000
X_test = np.ascontiguousarray(jetConstituent)[:njet]
Y_test = target[:njet]

In [ ]:
Y_predict = model.predict(X_test)
Y_hls = hls_model.predict(X_test)

In [ ]:
_, hls_trace = hls_model.trace(X_test)

In [ ]:
keras_trace = hls4ml.model.profiling.get_ymodel_keras(model, X_test)

In [ ]:
import matplotlib
#%matplotlib inline
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
for i, layer in enumerate(hls_trace.keys()):
    plt.figure()
    min_x = min(np.amin(hls_trace[layer]), np.amin(keras_trace[layer]))
    max_x = max(np.amax(hls_trace[layer]), np.amax(keras_trace[layer]))
    plt.plot([min_x, max_x], [min_x, max_x], c='gray')
    plt.scatter(hls_trace[layer].flatten(), keras_trace[layer].flatten(), s=0.2)
    plt.xlabel('hls4ml {}'.format(layer))
    plt.ylabel('QKeras {}'.format(layer))
    plt.show()
    plt.savefig('profiling_{:02d}_{}.png'.format(i, layer), dpi=300)

In [ ]:
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(15,5))
ax = plt.subplot(1, 2, 1)

# Plot the ROC curves
labels = ['gluon', 'quark', 'W', 'Z', 'top']
fpr = {}
tpr = {}
auc1 = {}
fpr_hls = {}
tpr_hls = {}
auc1_hls = {}
for i, label in enumerate(labels):
        fpr[label], tpr[label], threshold = roc_curve(Y_test[:,i], Y_predict[:,i])
        fpr_hls[label], tpr_hls[label], threshold_hls = roc_curve(Y_test[:,i], Y_hls[:,i])
        auc1[label] = auc(fpr[label], tpr[label])
        auc1_hls[label] = auc(fpr_hls[label], tpr_hls[label])
        ax.plot(tpr[label],fpr[label],label='%s tagger, auc = %.1f%%'%(label,auc1[label]*100.))
        ax.plot(tpr_hls[label],fpr_hls[label],label='%s tagger (hls4ml), auc = %.1f%%'%(label,auc1_hls[label]*100.),linestyle='dashed')
ax.semilogy()
ax.set_xlabel("sig. efficiency")
ax.set_ylabel("bkg. mistag rate")
ax.set_ylim(0.001,1)
ax.legend(loc='lower right')


# Plot DNN output 
ax = plt.subplot(1, 2, 2)
X = np.linspace(0.0, 1.0, 20)
hist={}
hist_hls={}
for i, name in enumerate(labels):
    hist[name] = ax.hist(Y_predict, bins=X, label=name ,histtype='step')
    hist_hls[name] = ax.hist(Y_hls, bins=X, label=name ,histtype='step', linestyle='dashed')
ax.semilogy()
ax.set_xlabel('DNN Output')
ax.legend(prop={'size': 10})
ax.legend(loc='lower left')
# Display plots
plt.show()
plt.savefig('results_hls4ml.pdf', dpi=300)
plt.savefig('results_hls4ml.png', dpi=300)